# Advanced Data Cleaning and Imputation
This notebook demonstrates advanced methods for handling missing values and outliers in the housing dataset, including structural filling, ordinal encoding, iterative imputation, and predictive imputation techniques.

In [19]:
# Import required libraries
import os
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error

In [20]:
# Load the training data
train_df = pd.read_csv('../../data/raw/train.csv')
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Inspect Missing Values
Check for missing values in the dataset to identify columns that need advanced imputation.

In [21]:
# Display missing value counts
missing = train_df.isnull().sum()
missing[missing > 0].sort_values(ascending=False)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
LotFrontage      259
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
BsmtFinType2      38
BsmtExposure      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
Electrical         1
dtype: int64

## Handling Structural Missing Values
Some features have missing values that indicate the absence of a feature (e.g., no garage, no alley access). These should be filled with a specific value like 'None' before advanced imputation.

In [22]:
# Fill structural missing values with 'None' for relevant categorical features
structural_none_features = ['Alley', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
                            'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                            'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']
for col in structural_none_features:
    if col in train_df.columns:
        train_df[col] = train_df[col].fillna('None')

In [ ]:
# Predictive imputation for a categorical column (e.g., GarageType)
feature = 'GarageType'
if train_df[feature].isnull().sum() > 0:
    not_null = train_df[train_df[feature].notnull()]
    null = train_df[train_df[feature].isnull()]
    predictors = [col for col in train_df.columns if col not in [feature, 'SalePrice']]
    X = pd.get_dummies(not_null[predictors], dummy_na=True)
    X_null = pd.get_dummies(null[predictors], dummy_na=True)
    X_null = X_null.reindex(columns=X.columns, fill_value=0)
    clf = RandomForestClassifier(n_estimators=100, random_state=0)
    clf.fit(X, not_null[feature])
    train_df.loc[train_df[feature].isnull(), feature] = clf.predict(X_null)


## Define Ordinal Features and Quality Mapping
Some features have a natural order (e.g., quality ratings). We'll define these before feature grouping to handle them appropriately.

In [23]:
# Define quality-related features and their order
quality_map = ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
ordinal_features = [
    'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC',
    'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC'
]
# Ensure all ordinal features are present in the data
ordinal_features = [col for col in ordinal_features if col in train_df.columns]
print(f"Ordinal features found: {ordinal_features}")

Ordinal features found: ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']


## Feature Grouping for Preprocessing
Split features into numeric, ordinal, and categorical groups for separate preprocessing pipelines.

In [24]:
# Identify numeric and categorical columns (excluding target and ordinal features)
numeric_features = train_df.select_dtypes(include=[np.number]).columns.drop('SalePrice', errors='ignore')
categorical_features = [col for col in train_df.select_dtypes(include='object').columns 
                       if col not in ordinal_features]
print(f"Numeric features: {len(numeric_features)}")
print(f"Ordinal features: {len(ordinal_features)}")
print(f"Categorical features: {len(categorical_features)}")

Numeric features: 37
Ordinal features: 10
Categorical features: 33


## Build Comprehensive Preprocessing Pipeline
Use scikit-learn's ColumnTransformer to apply different preprocessing strategies:
- **Numeric**: Iterative imputation + scaling
- **Ordinal**: Constant imputation + ordinal encoding
- **Categorical**: Constant imputation + one-hot encoding

In [ ]:
# Define preprocessing pipelines for each feature type
numeric_pipeline = Pipeline([
    ('imputer', IterativeImputer(
        estimator=RandomForestRegressor(n_estimators=50),
        max_iter=10, random_state=0)),
    ('scaler', StandardScaler())
])
ordinal_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='None')),
    ('ordinal', OrdinalEncoder(categories=[quality_map]*len(ordinal_features), handle_unknown='use_encoded_value', unknown_value=-1))
])
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='None')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])
# Combine all pipelines using ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_features),
    ('ord', ordinal_pipeline, ordinal_features),
    ('cat', categorical_pipeline, categorical_features)
])
# Fit and transform the data (excluding target)
X_clean = preprocessor.fit_transform(train_df.drop(columns='SalePrice'))
print(f"Preprocessed data shape: {X_clean.shape}")

Preprocessed data shape: (1460, 264)


## Advanced Predictive Imputation for Categorical Features
Use predictive modeling to fill missing categorical values (example for GarageType).

# Convert Preprocessed Data to DataFrame with Numeric Features
After preprocessing, the output is a NumPy array. To make it easier to work with, convert it back to a DataFrame with proper column names.

In [ ]:
# Convert preprocessed data to DataFrame with feature names
# Get feature names for each transformer
num_cols = list(numeric_features)
ord_cols = list(ordinal_features)
cat_cols = list(preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features))
all_cols = num_cols + ord_cols + list(cat_cols)

import scipy.sparse
if scipy.sparse.issparse(X_clean):
    X_clean_dense = X_clean.A
else:
    X_clean_dense = X_clean

X_clean_df = pd.DataFrame(X_clean_dense, columns=all_cols, index=train_df.index)
print(X_clean_df.head())
print(f"All features are now numeric. Shape: {X_clean_df.shape}")

         Id  MSSubClass  LotFrontage   LotArea  OverallQual  OverallCond  \
0 -1.730865    0.073375    -0.252202 -0.207142     0.651479    -0.517200   
1 -1.728492   -0.872563     0.391456 -0.091886    -0.071836     2.179628   
2 -1.726120    0.073375    -0.123471  0.073480     0.651479    -0.517200   
3 -1.723747    0.309859    -0.466755 -0.096897     0.651479    -0.517200   
4 -1.721374    0.073375     0.563098  0.375148     1.374795    -0.517200   

   YearBuilt  YearRemodAdd  MasVnrArea  BsmtFinSF1  ...  SaleType_ConLw  \
0   1.050994      0.878668    0.509962    0.575425  ...             0.0   
1   0.156734     -0.429577   -0.575140    1.171992  ...             0.0   
2   0.984752      0.830215    0.321730    0.092907  ...             0.0   
3  -1.863632     -0.720298   -0.575140   -0.499274  ...             0.0   
4   0.951632      0.733308    1.362543    0.463568  ...             0.0   

   SaleType_New  SaleType_Oth  SaleType_WD  SaleCondition_Abnorml  \
0           0.0        

## Save Cleaned Dataset
Export the cleaned dataset for further analysis or modeling.

In [39]:
# Save cleaned numeric data
os.makedirs('../../data/processed/V2', exist_ok=True)
X_clean_df.to_csv('../../data/processed/V2/train_advanced_cleaned.csv', index=False)
print(f"Cleaned numeric dataset saved with shape: {X_clean_df.shape}")

Cleaned numeric dataset saved with shape: (1460, 264)
